<a href="https://colab.research.google.com/github/jeongname/jeongname/blob/main/220315_hj_SANUP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install gluonnlp
# !pip install mxnet
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!pip install transformers
!pip install sentencepiece
!pip install kobert-transformers
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

     |████████████████████████████████| 344 kB 7.2 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595738 sha256=bc9b4c44d7be13b944e89a55c9272909a848cc1c6080b472646cb2ce408c7999
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-gg45s4hi/kobert-tokenizer_811aa12611a34cc3a9bf1c60360bc294
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-gg45s4hi/kobert-tokenizer_811aa12611a34cc3a9bf1c60360bc294
  Created wheel for kobert-tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4649 sha256=3ae962704aaab4f1eba864833104014862771990d3d905b153c2305988469a2e
  Stored in directory: /tmp/pip-ephem-wheel-cache-r9lwok1t/wheels/10/b4/d9/cb627bbfaefa266657b0b4e8127f7bf96d27376fa1a23897b4
Successfully built kobert-tokenizer
     |████████████████████████████████

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from kobert_tokenizer import KoBERTTokenizer
from sklearn.model_selection import train_test_split

In [6]:
df = pd.read_csv('/content/drive/MyDrive/SANUP/1. 실습용자료.txt', sep='|', encoding='cp949')
test = pd.read_csv('/content/drive/MyDrive/SANUP/2. 모델개발용자료.txt', sep='|', encoding='cp949')

In [7]:
df = df.fillna("")

In [8]:
df.isnull().sum()

AI_id        0
digit_1      0
digit_2      0
digit_3      0
text_obj     0
text_mthd    0
text_deal    0
dtype: int64

In [10]:
df["target"] = df["digit_1"]
df["text"] = df["text_obj"]+ " " + df["text_mthd"] + " " + df["text_deal"]

In [11]:
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,target,text
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환,S,카센터에서 자동차부분정비 타이어오일교환
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매,G,상점내에서 일반인을 대상으로 채소.과일판매
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매,G,절단하여사업체에도매 공업용고무를가지고 합성고무도매
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치,G,영업점에서 일반소비자에게 열쇠잠금장치
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육,Q,어린이집 보호자의 위탁을 받아 취학전아동보육
...,...,...,...,...,...,...,...,...,...
999995,id_0999996,C,13,134,제품입고,워싱,청바지워싱,C,제품입고 워싱 청바지워싱
999996,id_0999997,F,42,424,현장에서,고객의요청에의해,실내인테리어,F,현장에서 고객의요청에의해 실내인테리어
999997,id_0999998,G,47,474,영업점에서,일반소비자에게,여성의류 판매,G,영업점에서 일반소비자에게 여성의류 판매
999998,id_0999999,P,85,856,사업장에서,일반고객을대상으로,필라테스,P,사업장에서 일반고객을대상으로 필라테스


In [13]:
df['text'] = df['text'].str.strip()

In [16]:
train = df[['target','text']]

In [17]:
train.head()

,target,text
0,S,카센터에서 자동차부분정비 타이어오일교환
1,G,상점내에서 일반인을 대상으로 채소.과일판매
2,G,절단하여사업체에도매 공업용고무를가지고 합성고무도매
3,G,영업점에서 일반소비자에게 열쇠잠금장치
4,Q,어린이집 보호자의 위탁을 받아 취학전아동보육


In [18]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

Downloading:   0%|          | 0.00/363k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [20]:
def text_to_token(df:pd.DataFrame):
  input_ids = list()
  token_type_ids = list()
  attention_mask = list()

  for i in range(len(df)):
    temp = tokenizer.encode_plus(df['text'].loc[i], add_special_tokens=True,
            pad_to_max_length=True,
            return_attention_mask = True, 
            return_token_type_ids=True,
            truncation=True)
    input_ids.append(temp['input_ids'])
    token_type_ids.append(temp['token_type_ids'])
    attention_mask.append(temp['attention_mask'])

  return input_ids, token_type_ids, attention_mask

df['input_ids'], df['token_type_ids'], df['attention_mask'] = text_to_token(df)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [21]:
df[['input_ids', 'token_type_ids', 'attention_mask']].head()

,input_ids,token_type_ids,attention_mask
0,"[2, 4635, 6594, 3893, 6398, 6416, 7227, 6441, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,"[2, 2658, 7220, 5678, 6903, 3807, 7119, 7088, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,"[2, 4067, 5788, 7815, 6493, 6879, 6901, 6168, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,"[2, 3382, 7220, 6903, 3807, 6611, 6897, 3358, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
4,"[2, 3235, 2400, 7167, 3565, 7088, 2226, 4598, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"


In [22]:
## GPU
device = torch.device("cuda:0")

In [23]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [24]:
my_dict = {}
for i in range(len(df['target'].unique())):
  my_dict[df['target'].unique()[i]] = i

In [25]:
my_dict

{'A': 18,
 'B': 14,
 'C': 3,
 'D': 17,
 'E': 16,
 'F': 6,
 'G': 1,
 'H': 8,
 'I': 4,
 'J': 12,
 'K': 15,
 'L': 13,
 'M': 11,
 'N': 9,
 'O': 5,
 'P': 7,
 'Q': 2,
 'R': 10,
 'S': 0}

In [26]:
def target_to_label(text:str):
  return my_dict[text]

df['target_label'] = df['target'].map(target_to_label)

In [27]:
df[['target', 'target_label']].head()

,target,target_label
0,S,0
1,G,1
2,G,1
3,G,1
4,Q,2


In [28]:
df['No.']=df.index

In [30]:
# dataset_train = df[['No.', 'text', 'target_label']][:int(len(df)*0.8)]
# dataset_test = df[['No.', 'text', 'target_label']][int(len(df)*0.8):]
dataset_train, dataset_test = train_test_split(df[['No.', 'text', 'target_label']], test_size=0.2, random_state=42, shuffle=True)
dataset_train.to_csv('dataset_train.txt', index=False, header=True, sep='\t', encoding='utf8')
dataset_test.to_csv('dataset_test.txt', index=False, header=True, sep='\t', encoding='utf8')

In [31]:
dataset_train = nlp.data.TSVDataset('/content/dataset_train.txt', field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset('/content/dataset_test.txt', field_indices=[1,2], num_discard_samples=1)

In [33]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [34]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [48]:
## Setting parameters
max_len = 24
batch_size = 8
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [49]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [50]:
next(iter(data_train))

(array([   2, 2068, 7191, 6234, 2184, 1083, 7239, 4152,    3,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1], dtype=int32),
 array(9, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0], dtype=int32),
 3)

In [51]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [52]:
len(df['target_label'].unique())

19

In [53]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=520,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [54]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [55]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [56]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [57]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [58]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [59]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    # torch.cuda.empty_cache()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/100000 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 6.186196804046631 train acc 0.125
epoch 1 batch id 201 loss 6.482423782348633 train acc 0.010572139303482588
epoch 1 batch id 401 loss 6.180246353149414 train acc 0.009663341645885287
epoch 1 batch id 601 loss 6.140075206756592 train acc 0.009775374376039933
epoch 1 batch id 801 loss 6.162475109100342 train acc 0.009987515605493134
epoch 1 batch id 1001 loss 6.050621032714844 train acc 0.015484515484515484
epoch 1 batch id 1201 loss 5.98280143737793 train acc 0.025083263946711076
epoch 1 batch id 1401 loss 5.7646803855896 train acc 0.04032833690221271
epoch 1 batch id 1601 loss 5.580536365509033 train acc 0.05925983760149906
epoch 1 batch id 1801 loss 5.584099769592285 train acc 0.07717934480843976
epoch 1 batch id 2001 loss 5.330892562866211 train acc 0.09176661669165417
epoch 1 batch id 2201 loss 5.4548234939575195 train acc 0.10415720127214902
epoch 1 batch id 2401 loss 4.8871965408325195 train acc 0.11474385672636402
epoch 1 batch id 2601 loss 4.873305797576

In [ ]:
print(test_acc / (batch_id+1))